In [1]:
import subprocess
import pandas as pd
from io import StringIO
from fastavro import json_writer, parse_schema
from datetime import datetime

from fastavro import reader


import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

import json



In [3]:
path = 'D:/Master/BDMA/Courses/Semester_2/Big_Data_Management/Project/dataimporta/'

In [2]:
data = subprocess.run('hadoop fs -cat /temporal/peru/imp/x14200222.csv', capture_output=True, shell=True, encoding="latin-1")

In [3]:
data1 = data.stdout
df = pd.read_csv(StringIO(data1))
df

,"CADU,C,3","FANO,C,4","NDCL,C,6","FNUM,N,8,0","FEMB,N,8,0","FECH_RECEP,N,8,0","NDCLREG,C,6","FREG,N,8,0","FANOREG,C,4","CAGE,C,4",...,"VFOBSERDOL,N,11,3","VPESNET,N,14,3","VPESBRU,N,14,3","QUNIFIS,N,14,3","TUNIFIS,C,10","QUNICOM,N,14,3","TUNICOM,C,3","UBIGEO,C,6","DNOMCON,C,60","DDIRCON,C,80"
0,118,2022,10451,20220126,20220202,20220126,10451.0,20220216,2022.0,8700.0,...,35268.11,23000.000,23140.000,23000.0,KG,23000.000,KG,150201,NaN,NaN
1,235,2022,7707,20220205,20220207,20220205,7707.0,20220214,2022.0,6199.0,...,8138.90,74.797,79.636,539.0,U,539.000,U,150132,NaN,NaN
2,235,2022,7707,20220205,20220207,20220205,7707.0,20220214,2022.0,6199.0,...,3186.10,29.281,31.176,211.0,U,211.000,U,150132,NaN,NaN
3,235,2022,7707,20220205,20220207,20220205,7707.0,20220214,2022.0,6199.0,...,2326.75,21.383,22.767,205.0,U,205.000,U,150132,NaN,NaN
4,235,2022,7707,20220205,20220207,20220205,7707.0,20220214,2022.0,6199.0,...,2360.80,21.696,23.100,208.0,U,208.000,U,150132,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31659,118,2022,15615,20220208,20220214,20220208,15615.0,20220217,2022.0,7828.0,...,24053.64,2721.000,2721.000,2721.0,KG,2.721,TM,150124,NaN,NaN
31660,118,2022,15615,20220208,20220214,20220208,15615.0,20220217,2022.0,7828.0,...,32103.65,3587.000,3587.000,3587.0,KG,3.587,TM,150124,NaN,NaN
31661,118,2022,15615,20220208,20220214,20220208,15615.0,20220217,2022.0,7828.0,...,87000.00,15000.000,15000.000,15000.0,KG,15.000,TM,150124,NaN,NaN
31662,118,2022,15615,20220208,20220214,20220208,15615.0,20220217,2022.0,7828.0,...,26288.40,4612.000,4612.000,4612.0,KG,4.612,TM,150124,NaN,NaN


In [21]:
cols = list(df.columns)
fields = []
for col in cols:
    fields.append({"name":col, "type": "string"})

In [22]:
fields

[{'name': 'CADU,C,3', 'type': 'string'},
 {'name': 'FANO,C,4', 'type': 'string'},
 {'name': 'NDCL,C,6', 'type': 'string'},
 {'name': 'FNUM,N,8,0', 'type': 'string'},
 {'name': 'FEMB,N,8,0', 'type': 'string'},
 {'name': 'FECH_RECEP,N,8,0', 'type': 'string'},
 {'name': 'NDCLREG,C,6', 'type': 'string'},
 {'name': 'FREG,N,8,0', 'type': 'string'},
 {'name': 'FANOREG,C,4', 'type': 'string'},
 {'name': 'CAGE,C,4', 'type': 'string'},
 {'name': 'TDOC,C,1', 'type': 'string'},
 {'name': 'NDOC,C,11', 'type': 'string'},
 {'name': 'DNOMBRE,C,40', 'type': 'string'},
 {'name': 'CPAIDES,C,3', 'type': 'string'},
 {'name': 'CPUEDES,C,6', 'type': 'string'},
 {'name': 'CVIATRA,C,1', 'type': 'string'},
 {'name': 'CUNITRA,C,1', 'type': 'string'},
 {'name': 'CEMPTRA,C,4', 'type': 'string'},
 {'name': 'DMAT,C,30', 'type': 'string'},
 {'name': 'NCON,C,20', 'type': 'string'},
 {'name': 'CENTFIN,C,3', 'type': 'string'},
 {'name': 'CALM,C,4', 'type': 'string'},
 {'name': 'DNOMPRO,C,60', 'type': 'string'},
 {'name'

In [31]:
schema = {
    "doc": datetime.now().strftime("%m/%d/%Y, %H:%M:%S"),
    "name": "trade_item",
    "namespace": "peru_imports",
    "type": "record",
    "fields": fields
}
schema = json.dumps(schema)

In [24]:
schema

'{"doc": "03/31/2022, 21:47:45", "name": "trade_item", "namespace": "peru_imports", "type": "record", "fields": [{"name": "CADU,C,3", "type": "string"}, {"name": "FANO,C,4", "type": "string"}, {"name": "NDCL,C,6", "type": "string"}, {"name": "FNUM,N,8,0", "type": "string"}, {"name": "FEMB,N,8,0", "type": "string"}, {"name": "FECH_RECEP,N,8,0", "type": "string"}, {"name": "NDCLREG,C,6", "type": "string"}, {"name": "FREG,N,8,0", "type": "string"}, {"name": "FANOREG,C,4", "type": "string"}, {"name": "CAGE,C,4", "type": "string"}, {"name": "TDOC,C,1", "type": "string"}, {"name": "NDOC,C,11", "type": "string"}, {"name": "DNOMBRE,C,40", "type": "string"}, {"name": "CPAIDES,C,3", "type": "string"}, {"name": "CPUEDES,C,6", "type": "string"}, {"name": "CVIATRA,C,1", "type": "string"}, {"name": "CUNITRA,C,1", "type": "string"}, {"name": "CEMPTRA,C,4", "type": "string"}, {"name": "DMAT,C,30", "type": "string"}, {"name": "NCON,C,20", "type": "string"}, {"name": "CENTFIN,C,3", "type": "string"}, {"

In [32]:
schema = avro.schema.parse(schema)
writer = DataFileWriter(open(path+"version0.avro", "ub"), DatumWriter(), schema)

ValueError: invalid mode: 'ub'

In [27]:
writer.close()

In [13]:
records = df.to_dict('records')

In [33]:
load_hdfs = subprocess.run(['hadoop', 'fs', '-get', '/persistent/peru/imp/2022/version0.avro', path], capture_output=True, shell=True)


In [38]:
load_hdfs.returncode

0

In [14]:
for dicts in records:
    for keys in dicts:
        dicts[keys] = str(dicts[keys])

In [15]:
records

[{'CADU,C,3': '118',
  'FANO,C,4': '2022',
  'NDCL,C,6': '10451',
  'FNUM,N,8,0': '20220126',
  'FEMB,N,8,0': '20220202',
  'FECH_RECEP,N,8,0': '20220126',
  'NDCLREG,C,6': '10451.0',
  'FREG,N,8,0': '20220216',
  'FANOREG,C,4': '2022.0',
  'CAGE,C,4': '8700.0',
  'TDOC,C,1': '4',
  'NDOC,C,11': '20605152491',
  'DNOMBRE,C,40': '"MAMA FOODS S.A.C"',
  'CPAIDES,C,3': 'ES',
  'CPUEDES,C,6': 'ECPSJ',
  'CVIATRA,C,1': '1',
  'CUNITRA,C,1': 'nan',
  'CEMPTRA,C,4': '7770',
  'DMAT,C,30': 'nan',
  'NCON,C,20': 'LMM0346035',
  'CENTFIN,C,3': 'nan',
  'CALM,C,4': '3951.0',
  'DNOMPRO,C,60': 'MAMA FOODS S.A.C',
  'DDIRPRO,C,80': 'CAL. MIGUEL ALJOVIN NRO. 289 (ESPLADA DE HOSPITAL CASIMIRO ULLOA) LIMA - LIMA -',
  'DK,C,1': 'nan',
  'DK2,C,1': 'nan',
  'NSER,C,4': '1',
  'PART_NANDI,N,10,0': '713399100',
  'DCOM,C,100': 'PALLAR 920 SACOS DE 25 KG. C/U LARGE LIMA BEANS - DRIED BEANS / PALLAR EN GRANO 50/55',
  'DMER2,C,100': 'PALLAR 920 SACOS DE 25 KG. C/U LARGE LIMA BEANS - DRIED BEANS / PALLAR EN

In [16]:
for record in records:
    writer.append(record)

In [17]:
writer.close()

In [18]:
# 1. List to store the records
avro_records = []

# 2. Read the Avro file
with open('users.avro', 'rb') as fo:
    avro_reader = reader(fo)
    print(avro_reader.metadata)
    for record in avro_reader:
        avro_records.append(record)
        
# 3. Convert to pd.DataFrame
df_avro = pd.DataFrame(avro_records)

# Print the first couple of rows
df_avro.head()

{'avro.codec': 'null', 'avro.schema': '{"type": "record", "name": "trade_item", "namespace": "peru_imports", "fields": [{"type": "string", "name": "CADU,C,3"}, {"type": "string", "name": "FANO,C,4"}, {"type": "string", "name": "NDCL,C,6"}, {"type": "string", "name": "FNUM,N,8,0"}, {"type": "string", "name": "FEMB,N,8,0"}, {"type": "string", "name": "FECH_RECEP,N,8,0"}, {"type": "string", "name": "NDCLREG,C,6"}, {"type": "string", "name": "FREG,N,8,0"}, {"type": "string", "name": "FANOREG,C,4"}, {"type": "string", "name": "CAGE,C,4"}, {"type": "string", "name": "TDOC,C,1"}, {"type": "string", "name": "NDOC,C,11"}, {"type": "string", "name": "DNOMBRE,C,40"}, {"type": "string", "name": "CPAIDES,C,3"}, {"type": "string", "name": "CPUEDES,C,6"}, {"type": "string", "name": "CVIATRA,C,1"}, {"type": "string", "name": "CUNITRA,C,1"}, {"type": "string", "name": "CEMPTRA,C,4"}, {"type": "string", "name": "DMAT,C,30"}, {"type": "string", "name": "NCON,C,20"}, {"type": "string", "name": "CENTFIN,C,

,"CADU,C,3","FANO,C,4","NDCL,C,6","FNUM,N,8,0","FEMB,N,8,0","FECH_RECEP,N,8,0","NDCLREG,C,6","FREG,N,8,0","FANOREG,C,4","CAGE,C,4",...,"VFOBSERDOL,N,11,3","VPESNET,N,14,3","VPESBRU,N,14,3","QUNIFIS,N,14,3","TUNIFIS,C,10","QUNICOM,N,14,3","TUNICOM,C,3","UBIGEO,C,6","DNOMCON,C,60","DDIRCON,C,80"
0,118,2022,10451,20220126,20220202,20220126,10451.0,20220216,2022.0,8700.0,...,35268.11,23000.0,23140.0,23000.0,KG,23000.0,KG,150201,nan,nan
1,235,2022,7707,20220205,20220207,20220205,7707.0,20220214,2022.0,6199.0,...,8138.9,74.797,79.636,539.0,U,539.0,U,150132,nan,nan
2,235,2022,7707,20220205,20220207,20220205,7707.0,20220214,2022.0,6199.0,...,3186.1,29.281,31.176,211.0,U,211.0,U,150132,nan,nan
3,235,2022,7707,20220205,20220207,20220205,7707.0,20220214,2022.0,6199.0,...,2326.75,21.383,22.767,205.0,U,205.0,U,150132,nan,nan
4,235,2022,7707,20220205,20220207,20220205,7707.0,20220214,2022.0,6199.0,...,2360.8,21.696,23.1,208.0,U,208.0,U,150132,nan,nan


In [19]:
avro_records = []

reader = DataFileReader(open("users.avro", "rb"), DatumReader())
for record in reader:
        avro_records.append(record)
        
# 3. Convert to pd.DataFrame
df_avro = pd.DataFrame(avro_records)

# Print the first couple of rows
df_avro.head()

,"CADU,C,3","FANO,C,4","NDCL,C,6","FNUM,N,8,0","FEMB,N,8,0","FECH_RECEP,N,8,0","NDCLREG,C,6","FREG,N,8,0","FANOREG,C,4","CAGE,C,4",...,"VFOBSERDOL,N,11,3","VPESNET,N,14,3","VPESBRU,N,14,3","QUNIFIS,N,14,3","TUNIFIS,C,10","QUNICOM,N,14,3","TUNICOM,C,3","UBIGEO,C,6","DNOMCON,C,60","DDIRCON,C,80"
0,118,2022,10451,20220126,20220202,20220126,10451.0,20220216,2022.0,8700.0,...,35268.11,23000.0,23140.0,23000.0,KG,23000.0,KG,150201,nan,nan
1,235,2022,7707,20220205,20220207,20220205,7707.0,20220214,2022.0,6199.0,...,8138.9,74.797,79.636,539.0,U,539.0,U,150132,nan,nan
2,235,2022,7707,20220205,20220207,20220205,7707.0,20220214,2022.0,6199.0,...,3186.1,29.281,31.176,211.0,U,211.0,U,150132,nan,nan
3,235,2022,7707,20220205,20220207,20220205,7707.0,20220214,2022.0,6199.0,...,2326.75,21.383,22.767,205.0,U,205.0,U,150132,nan,nan
4,235,2022,7707,20220205,20220207,20220205,7707.0,20220214,2022.0,6199.0,...,2360.8,21.696,23.1,208.0,U,208.0,U,150132,nan,nan


In [20]:
date = json.loads(reader.schema)
date["doc"]


'03/31/2022, 14:33:22'

In [21]:
date["doc"]
reader.close()

In [22]:
parsed_schema = parse_schema(schema)

TypeError: unhashable type: 'RecordSchema'

In [ ]:
parse_schema

In [ ]:
records = df.to_dict('records')

In [ ]:
with open('file1.avro', 'wb') as out:
    json_writer(out, parsed_schema, records)
    out.close()

In [ ]:
# 1. List to store the records
avro_records = []

# 2. Read the Avro file
with open('file1.avro', 'rb') as fo:
    avro_reader = reader(fo)
    for record in avro_reader:
        avro_records.append(record)
        
# 3. Convert to pd.DataFrame
df_avro = pd.DataFrame(avro_records)

# Print the first couple of rows
df_avro.head()

In [ ]:
from fastavro import writer, parse_schema

schema = {
    'doc': 'A weather reading.',
    'name': 'Weather',
    'namespace': 'test',
    'type': 'record',
    'fields': [
        {'name': 'station', 'type': 'string'},
        {'name': 'time', 'type': 'long'},
        {'name': 'temp', 'type': 'int'},
    ],
}
parsed_schema = parse_schema(schema)

records = [
    {u'station': u'011990-99999', u'temp': 0, u'time': 1433269388},
    {u'station': u'011990-99999', u'temp': 22, u'time': 1433270389},
    {u'station': u'011990-99999', u'temp': -11, u'time': 1433273379},
    {u'station': u'012650-99999', u'temp': 111, u'time': 1433275478},
]

with open('weather.avro', 'wb') as out:
    writer(out, parsed_schema, records)

In [ ]:
# 1. List to store the records
avro_records = []

# 2. Read the Avro file
with open('weather.avro', 'rb') as fo:
    avro_reader = reader(fo)
    for record in avro_reader:
        avro_records.append(record)
        
# 3. Convert to pd.DataFrame
df_avro = pd.DataFrame(avro_records)

# Print the first couple of rows
df_avro.head()

In [3]:
# CLEAN FOLDERS INSIDE HADOOP

#PERU

subprocess.run(['hadoop', 'fs', '-rm', '-r', '/persistent/peru/exp/'], capture_output=True, shell=True)
subprocess.run(['hadoop', 'fs', '-mkdir', '/persistent/peru/exp/'], capture_output=True, shell=True)

subprocess.run(['hadoop', 'fs', '-rm', '-r', '/persistent/peru/imp/'], capture_output=True, shell=True)
subprocess.run(['hadoop', 'fs', '-mkdir', '/persistent/peru/imp/'], capture_output=True, shell=True)

subprocess.run(['hadoop', 'fs', '-rm', '-r', '/persistent/peru/metadata/'], capture_output=True, shell=True)
subprocess.run(['hadoop', 'fs', '-mkdir', '/persistent/peru/metadata/'], capture_output=True, shell=True)

#BRAZIL

subprocess.run(['hadoop', 'fs', '-rm', '-r', '/persistent/brazil/exp/'], capture_output=True, shell=True)
subprocess.run(['hadoop', 'fs', '-mkdir', '/persistent/brazil/exp/'], capture_output=True, shell=True)

subprocess.run(['hadoop', 'fs', '-rm', '-r', '/persistent/brazil/imp/'], capture_output=True, shell=True)
subprocess.run(['hadoop', 'fs', '-mkdir', '/persistent/brazil/imp/'], capture_output=True, shell=True)

subprocess.run(['hadoop', 'fs', '-rm', '-r', '/persistent/brazil/metadata/'], capture_output=True, shell=True)
subprocess.run(['hadoop', 'fs', '-mkdir', '/persistent/brazil/metadata/'], capture_output=True, shell=True)

#CHILE

subprocess.run(['hadoop', 'fs', '-rm', '-r', '/persistent/chile/exp/'], capture_output=True, shell=True)
subprocess.run(['hadoop', 'fs', '-mkdir', '/persistent/chile/exp/'], capture_output=True, shell=True)

subprocess.run(['hadoop', 'fs', '-rm', '-r', '/persistent/chile/imp/'], capture_output=True, shell=True)
subprocess.run(['hadoop', 'fs', '-mkdir', '/persistent/chile/imp/'], capture_output=True, shell=True)

subprocess.run(['hadoop', 'fs', '-rm', '-r', '/persistent/chile/metadata/imp'], capture_output=True, shell=True)
subprocess.run(['hadoop', 'fs', '-mkdir', '/persistent/chile/metadata/imp'], capture_output=True, shell=True)

subprocess.run(['hadoop', 'fs', '-rm', '-r', '/persistent/chile/metadata/exp'], capture_output=True, shell=True)
subprocess.run(['hadoop', 'fs', '-mkdir', '/persistent/chile/metadata/exp'], capture_output=True, shell=True)




CompletedProcess(args=['hadoop', 'fs', '-mkdir', '/persistent/chile/metadata/exp'], returncode=0, stdout=b'', stderr=b'')